In [23]:
import subprocess
import os
import datetime
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
from folium import plugins
import branca.colormap as cm
import matplotlib.pyplot as plt
import mplleaflet
from matplotlib.patches import Ellipse
import math

In [2]:
lat = widgets.FloatText(
    value=33.0,
    description='Lat',
)
lon = widgets.FloatText(
    value=-115.0,
    description='Lon',
)
width = widgets.FloatText(
    value=1,
    description='Width',
)
height = widgets.FloatText(
    value=1,
    description='Height')
t1 = widgets.DatePicker(description='Time 1',value = datetime.date(2010,4,7))
t2 = widgets.DatePicker(description='Time 2',value = datetime.date(2010,4,9))
file_name = widgets.Text(description='Output name', value='displacement')
all_widgets = [lat, lon, width, height, t1, t2, file_name]
widgets.GridBox(all_widgets, layout=widgets.Layout(
    grid_template_columns='repeat(2,400px)'))

GridBox(children=(FloatText(value=33.0, description='Lat'), FloatText(value=-115.0, description='Lon'), FloatT…

In [51]:
#disp_args = f'python getDisplacement.py --lat {lat.value} --lon {lon.value} --width {width.value} --height {height.value} -t1 {t1.value.strftime("%Y-%m-%d")} -t2 {t2.value.strftime("%Y-%m-%d")} -o {file_name.value}'
# parameters dict object
# required
parameters = {"lat":lat.value,"lon":lon.value,"width":width.value,"height":height.value,"epoch1":t1.value.strftime("%Y-%m-%d"),"epoch2":t2.value.strftime("%Y-%m-%d"),"output":file_name.value}
# optional
parameters['scale'] = None
parameters['ref'] = None 
parameters['ct'] = None 
parameters['pt'] = None
parameters['eon'] = True
parameters['mon'] = False
parameters['dwin1'] = None
parameters['dwin2'] = None
parameters['vabs'] = False

In [4]:
# turn dict object to a class object
class objdict(dict):

    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)


In [5]:
#process = subprocess.getoutput(disp_args)
from getDisplacement_fix import getDisplacement

In [52]:
data_table = getDisplacement(objdict(parameters))

In [53]:
datadf = pd.DataFrame(data_table, columns=["Site","Lon","Lat","Delta E","Delta N","Delta V","Sigma E","Sigma N","Sigma V"])

In [54]:
datadf

,Site,Lon,Lat,Delta E,Delta N,Delta V,Sigma E,Sigma N,Sigma V
0,BLYT,-114.714853,33.610414,0.160553,-1.401918,-0.409839,0.274003,0.324675,1.072682
1,BOMG,-115.729707,33.364628,-0.023444,-3.689764,0.705550,0.286649,0.342355,1.173154
2,CACT,-115.989952,33.655135,0.243172,-2.138514,-1.117248,0.285208,0.340873,1.139777
3,CRRS,-115.735045,33.069808,0.805165,-7.205936,1.144878,0.290387,0.349654,1.225760
4,DHLG,-115.788031,33.389806,-0.090734,-3.265312,1.162998,0.285348,0.339370,1.155894
5,GLRS,-115.521374,33.274811,0.454043,-4.707201,0.659400,0.289848,0.347003,1.201808
6,GMPK,-114.827347,33.051084,1.769741,-1.970751,-0.684817,0.313947,0.365123,1.364453
7,HNPS,-115.635338,33.705005,0.111806,-2.555877,0.611770,0.301482,0.373383,1.459064
8,IID2,-115.031804,32.706168,7.359603,-4.292746,-0.992672,0.297097,0.350955,1.247817
9,P003,-114.004954,32.722544,2.755570,0.210414,-1.562735,0.285556,0.338992,1.175968


In [49]:
def create_map(df, lon_col='Lon', lat_col='Lat', data_col='Delta V', site_col='Site'):
    '''
    Parameters
    ----------
    df :
        Pandas dataframe with column of lat, long, and a data column
    lon_col:
        Name of column with longitude coords. Default value of 'Lon'
    lat_col:
        Name of column with lattitude coords. Default value of 'Lat'
    data_col:
        Name of column with values to be colormapped. Default is 'Delta N'
    site_col:
        Name of column with GPS site names. 
    -------
    Returns:
        Folium map object
    '''
    start_cords = (df[lat_col].median(),
                   df[lon_col].median())
    my_map = folium.Map(start_cords,
                        tiles=None,zoom_start=8, width=750, height=465,contol_scale=True)
    plugins.Fullscreen(position='topleft').add_to(my_map)
    # load basemap with name
    folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                        attr='Tiles &copy; Esri', name='World_Topo_Map').add_to(my_map)

    
    # making MPL seismic colormap in Branca
    colors_list = ['blue', 'white', 'red']
    minimum = df[data_col].min()
    maximum = df[data_col].max()
    colormap = cm.LinearColormap(
        colors=colors_list, vmin=minimum, vmax=maximum)
    stations = folium.FeatureGroup(
        name='Vertical Displacement')
    horizontal = folium.FeatureGroup(
        name='Horizontal Displacement')
    scale = 320
    for cord, disp, site in zip(zip(df[lat_col], df[lon_col]), df[data_col], df[site_col]):
        folium.Circle(
            location=cord,
            radius=abs(disp)*scale,
            fill=True,
            color=colormap(disp),
            popup=f'Vertical Displacement:{"{:10.4f}".format(disp)} mm,   Site:{site}',
            stroke=True,
            weight=1,
            fill_opacity=.8
        ).add_to(stations)
    
    horizontal = folium.FeatureGroup(
        name='Horizontal Displacement')
    for lat,lon,vlon,vlat in zip(df[lat_col], df[lon_col],df['Delta E'],df['Delta N']):
        linedis = [[lat,lon],[lat+vlat/scale,lon+vlon/scale/math.cos(lat*math.pi/180.)]]
        folium.PolyLine(
            locations=linedis,
            color='red',
            weight=2
        ).add_to(horizontal)
                   
    errorlayer = folium.FeatureGroup(
        name='Error eclipse')
    for lat,lon,vlon,vlat,slon,slat in zip(df[lat_col], df[lon_col],df['Delta E'],df['Delta N'],df['Sigma E'],df['Sigma N']):
        theta = 0
        coords = []
        for k in range(0,31):
            angle = k/30*2*math.pi
            elon = slon*math.cos(angle)*math.cos(theta)-slat*math.sin(angle)*math.sin(theta)
            elat = slon*math.cos(angle)*math.sin(theta)+slat*math.sin(angle)*math.cos(theta)
            elon = (elon+vlon)/scale/math.cos(lat*math.pi/180.)
            elat = (elat+vlat)/scale
            coords.append([lat+elat,lon+elon])
        folium.Polygon(
            locations=coords,
            fill=False,
            color="black",
            weight=2 
        ).add_to(errorlayer)
    my_map.add_child(stations)
    my_map.add_child(horizontal)
    my_map.add_child(errorlayer)
    my_map.add_child(colormap)
#     folium.raster_layers.ImageOverlay('error_elippse.png', [[df[lat_col].min(
#     ), df[lon_col].min()], [df[lat_col].max(), df[lon_col].max()]], name='Error Ellipse').add_to(my_map)
    my_map.add_child(folium.LayerControl(postion='bottomleft'))
    
    return my_map

In [55]:
# I have no idea why the arrows are plotting off center from the circles. Seems super inconsistent. 
create_map(datadf)

In [31]:
pd

<module 'pandas' from '/home/cicuser/miniconda3/lib/python3.8/site-packages/pandas/__init__.py'>